# Step Functions

This document collates and describes the various step functions that are used for updating machine learning models. Throughout the notebook will be the examples and code for Lambda functions which will play certain parts in the updating process.

### Initialize

Firstly we call a Lambda function to initialise the 

#### Lambda Function

### List Models

Lists the machine learning models available in SageMaker.

#### Lambda Function

### If train

A choice function depending on whether the step function has chosen to build.

### Get training config

Gets the configuration for the training job.

#### Lambda Function

### Get data config

Takes confiruation of the data location.

#### Lambda Function

### If build training

### Build image training

#### Lambda Function

### Wait for training

### Build status training

Function that outputs the status of the training.

#### Lambda Function

### Check image training

### End build training

### If HPO

### Start HPO

### Wait for training

### Start training

Calls the Lambda function that starts the training job.

#### Lambda Function

### If HPO Status

### Get HPO Status

Invokes the Lambda function that gets the HPO Status

#### Lambda Function

### Get Training Status

Invokes the Lambda function that gets the Training Status

#### Lambda Function

### Check Training Status

### Check HPO Status

### Get Artifact

#### Lambda Function

### Get Model Config

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var sagemaker=new aws.SageMaker()

exports.handler=(event,context,callback)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
    if(event.params.ModelArtifacts){
        var ModelDataUrl=event.params.ModelArtifacts
    }else if(event.status.training.ModelArtifacts){
        var ModelDataUrl=event.status.training.ModelArtifacts.S3ModelArtifacts 
    }else{
        var ModelDataUrl=`${event.status.training.TrainingJobDefinition.OutputDataConfig.S3OutputPath}/${event.status.training.BestTrainingJob.TrainingJobName}/output/model.tar.gz`
    }

    try{
        callback(null,{
            ExecutionRoleArn:event.params.modelrole,
            ModelName:`${event.params.name}-${event.params.id}`,
            PrimaryContainer:{
                Image:event.params.InferenceImage || `${event.params.accountid}.dkr.ecr.${process.env.AWS_REGION}.amazonaws.com/${event.params.ecrrepo}:Inference_v${event.params.version}`,
                ModelDataUrl,
                Environment:event.params.modelhostingenvironment
            },
            Tags:[{
                Key:"sagebuild:stack",
                Value:event.params.stackname
            }]
        })
    }catch(e){
        callback(new Error(e))
    }
}

### If build Inference

### Build image Inference

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var codebuild=new aws.CodeBuild()

exports.handler=(event,context,cb)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
    codebuild.startBuild({
        projectName:event.params.projectname,
        environmentVariablesOverride:[{
            name:"IMAGE_TAG",
            value:`Inference_v${event.params.version}`
        },{
            name:"DOCKERFILE_PATH",
            value:event.params[`dockerfile_path_Inference`]
        }]
    }).promise()
    .then(result=>cb(null,result.build))
    .catch(x=>cb(new Error(x)))
}

### Wait Inference

### Build Status Inference

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var codebuild=new aws.CodeBuild()

exports.handler=(event,context,cb)=>{
    console.log(JSON.stringify(event,null,2))
    codebuild.batchGetBuilds({
        ids:[event.outputs.build.Inference.id] 
    }).promise()
    .then(result=>cb(null,result.builds[0]))
    .catch(x=>{
        console.log(x)
        cb(new Error(x))
    })
}

### Check image Inference

### End Build Inference

### Create Model

#### Lambda Function

### Get Endpoint Config

#### Lambda Function

### Create Endpoint Config

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var sagemaker=new aws.SageMaker()

exports.handler=(event,context,cb)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
    
    var args=Object.assign({
        EndpointConfigName:`${event.params.name}-${event.params.id}`,
        Tags:[]
    },event.args.endpoint) 
    
    if(event.params.models[0]){
        args.Tags.push({
            Key:"sagebuild:previous",Value:event.params.models[0]
        })
    }
    sagemaker.createEndpointConfig(args).promise()
    .then(result=>{
        cb(null,result)
    })
    .catch(x=>cb(new Error(x)))
}

### Update Endpoint

#### Lambda Function

### Wait for Endpoint

### Endpoint Status

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var sagemaker=new aws.SageMaker()

exports.handler=(event,context,cb)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
    try{ 
        sagemaker.describeEndpoint({
            EndpointName:event.params.endpointname
        }).promise()
        .then(result=>{
            cb(null,result)
        })
        .catch(function(error){
            console.log(error)
            if(error.message.match(/Could not find endpoint/)){
                cb(null,{EndpointStatus:"Empty"})
            }else{
                cb(new Error(error))
            }
        })
    }catch(e){
        console.log(e)
        cb(new Error(e))
    }
}

### Endpoint Check

### Deploy Status

#### Lambda Function

### Deploy Check

### Success

#### Lambda Function

### Fail

## Delete Group

### Should Delete Endpoint

### Endpoint Delete

#### Lambda Function

### Endpoint Delete Status

#### Lambda Function

### Wait for Delete

### Endpoint Delete Check

### End Delete

### Delete Endpoint Config

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var sagemaker=new aws.SageMaker()

exports.handler=(event,context,cb)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
    var name=event.outputs.endpoint.EndpointConfigArn.match(
            /arn:aws:sagemaker:.*:.*:endpoint-config\/(.*)/)[1]
   
    sagemaker.deleteEndpointConfig({
        EndpointConfigName:name
    }).promise()
    .then(result=>{
        cb(null,result)
    })
    .catch(x=>cb(new Error(x)))
}

### Delete Model

#### Lambda Function

### All deletes caught by error

### Endpoint Fail

#### Lambda Function

### Fail build notification Inference

#### Lambda Function

### Fail build notification Training

#### Lambda Function

In [ ]:
var aws=require('aws-sdk')
aws.config.region=process.env.AWS_REGION 
var sns=new aws.SNS()
var result="Failed"
exports.handler=(event,context,cb)=>{
    console.log("EVENT:",JSON.stringify(event,null,2))
  
    sns.publish({
        TopicArn:event.params.statustopic,
        Subject:`SageBuild ${result}`,
        Message:JSON.stringify({
            Result:"Fail",
            StackName:event.params.stackname,
            Name:event.params.name,
            Date:new Date(),
            Error:event.error
        },null,2),
        MessageAttributes:{
            event:{
                DataType:"String",
                StringValue:JSON.stringify(event)
            }
        }
    }).promise()
    .then(result=>{
        cb(null,{})
    })
    .catch(cb)
}